In [3]:
# add any other imports you need here!
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt

In [4]:
# use this to compute distance, not Euclidean Distance!
def haversine(lat1, lon1, lat2, lon2):

      R = 3959.87433 # this is in miles.  For Earth radius in kilometers use 6372.8 km

      dLat = radians(lat2 - lat1)
      dLon = radians(lon2 - lon1)
      lat1 = radians(lat1)
      lat2 = radians(lat2)

      a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
      c = 2*asin(sqrt(a))

      return R * c

# Optimal Center Formula
- parameters:
1. county_lat: a double containing the latidude of the county's population center
2. county_long: a double containing the longitude of the county's population center
3. potential_center_lat: a double containing the latidude of the potential center
 potential_center_long: a double containing the longitude of the county's potential center
 county_classification: an int from 1-9 representing how much mental health need is required (9 = high need, 1 = low need)
- return: should return a double representing a score of how optimal this center is


In [5]:
# params:
# county_lat: a double containing the latidude of the county's population center
# county_long: a double containing the longitude of the county's population center
# potential_center_lat: a double containing the latidude of the potential center
# potential_center_long: a double containing the longitude of the county's potential center
# county_classification: an int from 1-9 representing how much mental health need is required (9 = high need, 1 = low need)
# return: should return a double representing a score of how optimal this center is

def optimal_center_formula(county_lat, county_long, potential_center_lat, potential_center_long, county_classification):
    distance = haversine(county_lat, county_long, potential_center_lat, potential_center_long)
    # perform some mathematical operation with this calculated distance and the county classification

In [ ]:
def test_optimal_center_formula():
    # testcase 1: should return a high value, since coordinates are close and need is high
    
    # testcase 2: should return a low value, since coordinates are far and need is low
    
    # testcase 3: should return a moderate-low value, since coordinates are far even though need is high
    
    # testcase 4: should return a moderate value, since coordinates are close and need is moderate-low